In [3]:
from dotenv import load_dotenv
import os
import requests
import json

In [4]:

class SwaggerFetcher:
    def __init__(self):
        load_dotenv()
        self.urls = self.get_urls_from_env()
    
    def get_urls_from_env(self):
        urls_string = os.getenv('SWAGGER_URLS', '')
        return [url.strip() for url in urls_string.split(',') if url.strip()]
    
    def fetch_all(self):
        results = []
        for url in self.urls:
            result = self.fetch_single(url)
            results.append(result)
        return results
    
    def fetch_single(self, url):
        try:
            response = requests.get(url, timeout=30)
            return {
                'url': url,
                'success': response.status_code == 200,
                'data': response.json() if response.status_code == 200 else None,
                'error': None
            }
        except Exception as e:
            return {
                'url': url,
                'success': False, 
                'data': None,
                'error': str(e)
            }

# الاستخدام
fetcher = SwaggerFetcher()
all_data = fetcher.fetch_all()

# طباعة النتائج
for item in all_data:
    status = "✅" if item['success'] else "❌"
    print(f"{status} {item['url']}")

✅ https://erpbackendpro.maids.cc/accounting/v3/api-docs
✅ https://erpbackendpro.maids.cc/recruitment/v3/api-docs
✅ https://erpbackendpro.maids.cc/clientmgmt/v3/api-docs
✅ https://erpbackendpro.maids.cc/payroll/v3/api-docs
✅ https://erpbackendpro.maids.cc/staffmgmt/v3/api-docs
✅ https://erpbackendpro.maids.cc/visa/v3/api-docs
✅ https://erpbackendpro.maids.cc/sales/v3/api-docs


In [5]:
print(json.dumps(all_data, indent=2, ensure_ascii=False)[:2000])


[
  {
    "url": "https://erpbackendpro.maids.cc/accounting/v3/api-docs",
    "success": true,
    "data": {
      "openapi": "3.0.3",
      "info": {
        "title": "Maids.cc API",
        "description": "API documentation for Magnamedia modules",
        "contact": {
          "name": "Muhannad",
          "email": "\nmohannad@maids.cc"
        },
        "version": "swagger"
      },
      "servers": [
        {
          "url": "https://erpbackendpro.maids.cc:443",
          "description": "Inferred Url"
        }
      ],
      "tags": [
        {
          "name": "client-refund-todo-controller",
          "description": "Client Refund Todo Controller"
        },
        {
          "name": "contract-controller",
          "description": "Contract Controller"
        },
        {
          "name": "contract-payment-confirmation-to-do-controller",
          "description": "Contract Payment Confirmation To Do Controller"
        },
        {
          "name": "direct-debit-contro

In [7]:


import chromadb

client = chromadb.PersistentClient(path="./my_chroma")
collection = client.get_or_create_collection(name="test_collection")

In [8]:
import chromadb

# إنشاء ChromaDB
client = chromadb.PersistentClient(path="./my_chroma")
collection = client.get_or_create_collection(name="swagger_apis")

# إضافة البيانات من all_data
documents = []
ids = []
metadatas = []

for i, api in enumerate(all_data):
    if api['success'] and api['data']:
        swagger_data = api['data']
        api_name = swagger_data.get('info', {}).get('title', f'API_{i}')
        
        # استخراج endpoints
        for path, methods in swagger_data.get('paths', {}).items():
            for method, details in methods.items():
                if method.lower() in ['get', 'post', 'put', 'delete']:
                    
                    # النص البسيط
                    text = f"{method.upper()} {path} - {details.get('summary', 'No description')}"
                    
                    documents.append(text)
                    ids.append(f"{i}_{method}_{path}".replace('/', '_').replace('{', '').replace('}', ''))
                    metadatas.append({
                        "api": api_name,
                        "method": method.upper(),
                        "path": path
                    })

# إضافة لـ ChromaDB
collection.add(documents=documents, ids=ids, metadatas=metadatas)

print(f"✅ تم إضافة {len(documents)} endpoint")
print(f"📊 إجمالي البيانات: {collection.count()}")

C:\Users\ZA\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [04:58<00:00, 278kiB/s]    


✅ تم إضافة 198 endpoint
📊 إجمالي البيانات: 198


In [ ]:


from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from chroma_search_tool import ChromaSearchTool
import os
from dotenv import load_dotenv

load_dotenv()

class AgentState(TypedDict):
    messages: Annotated[List, operator.add]

class SwaggerReactAgent:
    
    def __init__(self, model_name="gpt-4o-mini"):
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY مطلوب في ملف .env")
        
        # إنشاء الـ LLM
        self.llm = ChatOpenAI(
            model=model_name,
            temperature=0,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        
        self.tools = [ChromaSearchTool()]
        
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        self.tool_node = ToolNode(self.tools)
        self.graph = self._create_graph()
        
        print("✅ تم إنشاء SwaggerReactAgent بنجاح!")
    
    def _create_graph(self):
        
        workflow = StateGraph(AgentState)
        
        workflow.add_node("agent", self.agent_node)
        workflow.add_node("tools", self.tool_node)
        
        workflow.set_entry_point("agent")
        
        workflow.add_conditional_edges(
            "agent",
            self.should_continue,
            {
                "continue": "tools",
                "end": END
            }
        )
        
        workflow.add_edge("tools", "agent")
        
        return workflow.compile()
    
    def agent_node(self, state: AgentState):
        
        messages = state["messages"]
        
        system_prompt = """أنت مساعد ذكي متخصص في Swagger APIs. مهمتك الإجابة على أسئلة المستخدمين حول APIs المتاحة.

🎯 **دورك:**
- فهم أسئلة المستخدمين حول APIs
- استخدام أداة البحث للعثور على المعلومات ذات الصلة
- تقديم إجابات واضحة ومفيدة مع أمثلة عملية

💡 **كيفية العمل:**
1. عندما يسأل المستخدم عن شيء، استخدم أداة chroma_search للبحث
2. حلل النتائج واختر الأنسب للسؤال
3. اشرح للمستخدم كيفية استخدام API مع أمثلة
4. إذا لم تجد معلومات كافية، جرب البحث بكلمات مفتاحية مختلفة

📝 **نصائح الإجابة:**
- كن واضح ومفيد
- اذكر HTTP method والـ path
- اعطي أمثلة عملية إذا أمكن
- إذا كان السؤال غامض، اطلب توضيح"""

        full_messages = [
            {"role": "system", "content": system_prompt}
        ] + [
            {"role": msg.type if hasattr(msg, 'type') else "user", "content": msg.content}
            for msg in messages
        ]
        
        response = self.llm_with_tools.invoke(full_messages)
        
        return {"messages": [response]}
    
    def should_continue(self, state: AgentState):
        
        messages = state["messages"]
        last_message = messages[-1]
        
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "continue"
        else:
            return "end"
    
    def ask(self, question: str, verbose=True):
        
        if verbose:
            print(f"🤔 السؤال: {question}")
            print("🔄 الـ Agent يفكر...")
        
        # إنشاء initial state
        initial_state = {
            "messages": [HumanMessage(content=question)]
        }
        
        try:
            # تشغيل الـ graph
            result = self.graph.invoke(initial_state)
            
            # استخراج الإجابة النهائية
            final_message = result["messages"][-1]
            
            if hasattr(final_message, 'content'):
                response = final_message.content
            else:
                response = str(final_message)
            
            if verbose:
                print(f"🤖 الإجابة:")
                print(response)
                print("-" * 60)
            
            return response
            
        except Exception as e:
            error_msg = f"❌ حدث خطأ: {str(e)}"
            if verbose:
                print(error_msg)
            return error_msg

# دالة للاختبار السريع
def test_agent():
    
    print("🧪 اختبار SwaggerReactAgent...")
    print("=" * 60)
    
    try:
        agent = SwaggerReactAgent()
        
        # أسئلة تجريبية
        test_questions = [
            "شو هي برامترات API documentation for Magnamedia modules"

        ]
        
        for i, question in enumerate(test_questions, 1):
            print(f"\n🔢 سؤال {i}:")
            agent.ask(question)
            
            if i < len(test_questions):
                input("⏸️ اضغط Enter للسؤال التالي...")
        
        print("\n🎉 انتهى الاختبار!")
        
    except Exception as e:
        print(f"❌ خطأ في الاختبار: {e}")
        print("💡 تأكد من:")
        print("1. وجود OPENAI_API_KEY في ملف .env")
        print("2. وجود ChromaDB مع بيانات")

if __name__ == "__main__":
    test_agent()

ImportError: cannot import name 'get_tracing_context' from 'langsmith.run_helpers' (c:\Users\ZA\anaconda3\envs\training2\lib\site-packages\langsmith\run_helpers.py)

In [ ]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langgraph.graph import StateGraph, START, END
graph_builder = StateGraph(State)
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from config import  LLMConfig, load_system_prompt

llm = ChatOpenAI(temperature=LLMConfig.TEMPERATURE, model=LLMConfig.MODEL_NAME)
systemPrompt = load_system_prompt()

tools = toolkit.get_tools()